In [1]:
library(Seurat)
library(infercnv)
options(jupyter.plot_scale=1, future.globals.maxSize= 200*1024^3)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




In [2]:
seu <- readRDS('data/b16_5batch.rds')
all_types <- as.character(unique(seu$type))
use_cells <- c(seu$type[seu$type=='Mela'], sample(seu$type[seu$type!='Mela'], size=30000))
seu <- subset(seu, cells=names(use_cells))

In [3]:
cellAnnotation <- data.frame(seu$type)
rownames(cellAnnotation) <- colnames(seu)
table(cellAnnotation)

seu.type
         B Macro/Mono          T       Mela        Neu         NK       cDC2 
      4250      12870       6893       7066       1920       1205        917 
     MigDC       cDC1        pDC      Eryth        Tgd      Fibro 
       776        504        341        194        113         17 

In [4]:
infercnv_obj = CreateInfercnvObject(raw_counts_matrix=GetAssayData(seu, layer="counts"),
                                    annotations_file=cellAnnotation,
                                    delim="\t",
                                    gene_order_file="../yu/mouse_gencode.GRCm38.p6.vM25.basic.annotation.by_gene_name.infercnv_positions",
                                    ref_group_names=all_types[all_types!='Mela'])

INFO [2025-06-20 16:30:57] Parsing gene order file: ../yu/mouse_gencode.GRCm38.p6.vM25.basic.annotation.by_gene_name.infercnv_positions
INFO [2025-06-20 16:30:57] ::order_reduce:Start.
INFO [2025-06-20 16:30:59] .order_reduce(): expr and order match.
INFO [2025-06-20 16:31:07] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 32285,37066 Total=282972801 Min=0 Max=15064.
INFO [2025-06-20 16:31:08] num genes removed taking into account provided gene ordering list: 2195 = 6.79882298280935% removed.
INFO [2025-06-20 16:31:08] -filtering out cells < 100 or > Inf, removing 0 % of cells
WARN [2025-06-20 16:31:10] Please use "options(scipen = 100)" before running infercnv if you are using the analysis_mode="subclusters" option or you may encounter an error while the hclust is being generated.
INFO [2025-06-20 16:31:20] validating infercnv_obj


In [5]:
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=0.1, # cutoff=1 works well for Smart-seq2, and cutoff=0.1 works well for 10x Genomics
                             out_dir='infercnv_b16f10', 
                             cluster_by_groups=TRUE, 
                             denoise=TRUE,
                             HMM=TRUE)

INFO [2025-06-20 16:31:20] ::process_data:Start
INFO [2025-06-20 16:31:20] Creating output path infercnv_b16f10
INFO [2025-06-20 16:31:20] Checking for saved results.
INFO [2025-06-20 16:31:20] 

	STEP 1: incoming data

INFO [2025-06-20 16:33:37] 

	STEP 02: Removing lowly expressed genes

INFO [2025-06-20 16:33:37] ::above_min_mean_expr_cutoff:Start
INFO [2025-06-20 16:33:37] Removing 22306 genes from matrix as below mean expr threshold: 0.1
INFO [2025-06-20 16:33:40] validating infercnv_obj
INFO [2025-06-20 16:33:40] There are 7784 genes and 37066 cells remaining in the expr matrix.


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.1 GiB”


INFO [2025-06-20 16:34:11] no genes removed due to min cells/gene filter
INFO [2025-06-20 16:36:29] 

	STEP 03: normalization by sequencing depth

INFO [2025-06-20 16:36:29] normalizing counts matrix by depth


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.1 GiB”


INFO [2025-06-20 16:37:03] Computed total sum normalization factor as median libsize: 3124.000000
INFO [2025-06-20 16:37:04] Adding h-spike
INFO [2025-06-20 16:37:04] -hspike modeling of Macro/Mono
INFO [2025-06-20 16:41:18] -hspike modeling of NK
INFO [2025-06-20 16:45:18] -hspike modeling of cDC2
INFO [2025-06-20 16:50:21] -hspike modeling of MigDC
INFO [2025-06-20 16:54:09] -hspike modeling of Neu
INFO [2025-06-20 16:57:57] -hspike modeling of B
INFO [2025-06-20 17:02:39] -hspike modeling of T
INFO [2025-06-20 17:06:45] -hspike modeling of cDC1
INFO [2025-06-20 17:11:09] -hspike modeling of Tgd
INFO [2025-06-20 17:15:15] -hspike modeling of pDC
INFO [2025-06-20 17:19:27] -hspike modeling of Eryth
INFO [2025-06-20 17:24:08] -hspike modeling of Fibro
INFO [2025-06-20 17:28:31] validating infercnv_obj
INFO [2025-06-20 17:28:31] normalizing counts matrix by depth
INFO [2025-06-20 17:28:32] Using specified normalization factor: 3124.000000
INFO [2025-06-20 17:31:07] 

	STEP 04: log trans

...

INFO [2025-06-20 18:33:21] define_signif_tumor_subclusters(), tumor: pDC
INFO [2025-06-20 18:33:21] Setting auto leiden resolution for pDC to 0.0564516


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  Tdp2, BC005537, Acot13, Mrs2, Gmnn, Ripor2, Sox4, Cdkal1, Cmah, E2f3 
	   Uqcrfs1, Dusp22, Exoc2, Gmds, Hfe, Wrnip1, Mctp2, Chd2, 1810026B05Rik, Serpinb1a 
	   Fam174b, Serpinb6b, Arrdc4, Abt1, Slco3a1, Serpinb9, Akap13, Serpinb6a, Nqo2, Fmnl2 
Negative:  Fbxl20, Rpl19, Med1, Arl5c, Cdk12, Stard3, Mien1, Ikzf3, Lasp1, Ormdl3 
	   Psmd3, Rpl23, Msl1, Casc3, Cwc25, Wipf2, Rara, Top2a, Pip4k2b, Igfbp4 
	   Ccr7, Psmb3, Smarce1, Eif1, Pcgf2, Nt5c3b, Cisd3, Acly, Mllt6, Cnp 
PC_ 2 
Positive:  Csnk1g1, Pclaf, Trip4, Ppib, Zfp609, Snx1, Ciao2a, Rps27l, Rab8b, Oaz2 
	   Lactb, Herc1, Aph1c, Plekho2, Usp3, Tpm1, Spg21, Pdcd7, Vps13c, Clpx 
	   Dpp8, Ints14, Rora, Hacd3, Dennd4a, Rab11a, Tipin, Anxa2, Map2k1, Bnip2 
Negative:  Nlrc5, Herpud1, Nup93, Cpne2, Mt1, Fam192a, Mt2, Rspry1, Nudt21, Arl2bp 
	   Ccl22, Amfr, Ciapin1, Lpcat2, Co

INFO [2025-06-20 18:33:26] define_signif_tumor_subclusters(), tumor: Eryth
INFO [2025-06-20 18:33:26] Setting auto leiden resolution for Eryth to 0.0916085


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  Dhx16, 2310061I04Rik, Ppp1r18, Mrps18b, Nrm, Tubb5, Mdc1, Ppp1r10, Flot1, Abcf1 
	   Ier3, Gnl1, Gtf2h4, A930015D03Rik, H2-Q7, H2-Q6, H2-T23, H2-D1, H2-Q4, H2-T22 
	   Ddx39b, Nfkbil1, Gm11127, Rpl18, Cyth2, Sphk2, Grwd1, Bcat2, Tnf, Gm7030 
Negative:  Bpgm, Akr1b10, Akr1b8, Akr1b3, Cald1, Cyren, Slc35b4, Wdr91, Exoc4, Tmem209 
	   Chchd3, Copg2, Klhdc10, Zc3hc1, Lncpint, Cnot4, Mkln1, 2210408F21Rik, Ube2h, Nrf1 
	   Ahcyl2, Nup205, Tnpo3, Irf5, 1810058I24Rik, Atp6v1f, Calu, Usp4, Mtpn, Hilpda 
PC_ 2 
Positive:  Map4, Dhx30, Smarcc1, Scap, Cdc25a, Nme6, Ptpn23, Klhl18, Tma7, Atrip 
	   Setd2, Shisa5, Nbeal2, Uqcrc1, Ccdc12, Ccrl2, Lrrfip2, Pdcd6ip, Clasp2, Arih2 
	   Ubp1, Wdr6, Crtap, Glb1, Ndufaf3, Cnot10, Dync1li1, Impdh2, Cmtm6, Cmtm7 
Negative:  Rbpj, Smim20, Anapc4, Stim2, Pcdh7, Arap2, Pi4k2b, Rell1, Pgm2, Dhx15 
	   

INFO [2025-06-20 18:33:28] define_signif_tumor_subclusters(), tumor: Fibro
INFO [2025-06-20 18:33:28] Less cells in group Fibro than k_nn setting. Keeping as a single subcluster.
INFO [2025-06-20 18:33:28] -mirroring for hspike
INFO [2025-06-20 18:33:28] define_signif_tumor_subclusters(p_val=0.1
INFO [2025-06-20 18:33:29] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_Macro/Mono
INFO [2025-06-20 18:33:29] cut tree into: 1 groups
INFO [2025-06-20 18:33:29] -processing spike_tumor_cell_Macro/Mono,spike_tumor_cell_Macro/Mono_s1
INFO [2025-06-20 18:33:29] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_NK
INFO [2025-06-20 18:33:29] cut tree into: 1 groups
INFO [2025-06-20 18:33:29] -processing spike_tumor_cell_NK,spike_tumor_cell_NK_s1
INFO [2025-06-20 18:33:29] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_cDC2
INFO [2025-06-20 18:33:29] cut tree into: 1 groups
INFO [2025-06-20 18:33:29] -processing spike_tumor_cell_cDC2,spike_tumor_cell_cDC2_s1
INFO [2